# Translator

Данная тетрадка предназначена для перевода текста. В качестве переводчика будет использоваться Yandex Translator. Настройку подключения производили через YandexCloud. Для доступа используется сервисный аккаунт с API ключом. Запросы отправляются к веб сервису при помощи модуля request. Подробнее о настройке можно почитать [здесь](https://cloud.yandex.ru/docs/translate/quickstart).

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

import os
from dotenv import load_dotenv

import requests
import time
import json

import logging
from datetime import datetime

In [2]:
PATH_LYRICS = 'data/raw/lyrics.json'
PATH_META = 'data/raw/meta.json'
PATH_TO_SAVE_LYRICS = 'data/preprocessing/translated_lyrics.csv'
PATH_TO_SAVE_TITLE = 'data/preprocessing/translated_title.csv'

Для запросов в файле .env необходимо задать переменную YANDEX_API_KEY и присвоить ей значение API ключа полученного на YandexCloud.

In [3]:
load_dotenv('.env')
api_key = os.getenv('YANDEX_API_KEY')

Напишем функцию для перевода текста.

In [4]:
def translate_text(texts: str, target_language: str, api_key: str) -> str:
    body = {
        "targetLanguageCode": target_language,
        "texts": texts
    }

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Api-Key {api_key}"
    }

    response = requests.post('https://translate.api.cloud.yandex.net/translate/v2/translate',
        json = body,
        headers = headers
    )

    text = json.loads(response.text)['translations'][0]['text']
    return text

Напишем функцию для получения языка текста, если он отсутствует (он потребуется, чтобы не переводить и так переведённые тексты плюс в дальнейшем может выступить дополнительным признаком).

In [5]:
def detect_language_text(text: str, api_key: str) -> str:
    body = {
        "text": text[:999] #лимит на длину символов не более 1000
    }

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Api-Key {api_key}"
    }

    response = requests.post('https://translate.api.cloud.yandex.net/translate/v2/detect',
        json = body,
        headers = headers
    )

    language = json.loads(response.text)['languageCode']
    return language.upper()

Создадим функцию, которая будет переводить датафрейм. При этом необходимо учесть, что у переводчика есть квота обрабатываемых запросов в секунду и количество обрабатываемых символов в час, поэтому данные ситуации необходимо обработать. Так же есть общий лимит на количество переводимой информации, поэтому предусмотрим выход из функции, с возвращением обработанной информации. Для отслеживания прогресса будем использовать логирование с помощью модуля logging

In [25]:
def translate_df(df: pd.DataFrame, 
                 text_columns: str, 
                 target_language: str, 
                 api_key: str, 
                 verbose: int = 500) -> pd.DataFrame:
    log = logging.getLogger()
    log.setLevel(logging.INFO)
    logging.info(f"{datetime.now().time()} начат перевод датасета")
    
    df = df.copy(deep=True).reset_index(drop=True)
    df['translate_text'] = ''
    
    for i in range(df.shape[0]):
        texts = df.loc[i, text_columns]
        
        if df.loc[i, 'language'] is None or df.loc[i, 'language']  is np.NaN:
            try:
                language = detect_language_text(texts, api_key)
            except:
                # если превысили квоту запросов в секунду, ждём 1,1 секунды
                time.sleep(1.1)
                try:
                    language = detect_language_text(texts, api_key)
                except:
                    language = None
            df.loc[i,'language'] = language
            
        if df.loc[i, 'language'] is not None and df.loc[i,'language'].lower() == target_language:            
            translated_text = texts
            
        else:
            if texts is None:
                translated_text = texts
            else:
                try:
                    translated_text = translate_text(texts, target_language, api_key)
                except:
                    # если превысили квоту запросов в секунду, ждём 1,1 секунды
                    time.sleep(2)
                    try:
                        translated_text = translate_text(texts, target_language, api_key)
                    except:
                        # если превысили квоту запросов в час, ждём 1 час и 1 минуту
                        logging.info(f'''{datetime.now().time()} обработано {i} строки, превышено количество запросов в час,
                                      начато ожидание''')
                        time.sleep(3660)
                        try:
                            translated_text = translate_text(texts, target_language, api_key)
                        except:
                            # если превысили общую квоту запросов
                            logging.info(f'''{datetime.now().time()} обработано {i} строки, превышено количество запросов 
                            обработка датасета прекращена''')
                            return df        
        df.loc[i, 'translate_text'] = translated_text
        
        if verbose is not None:
            if i % verbose == 0:
                logging.info(f"{datetime.now().time()} обработано {i + 1} строк")
        
    logging.info(f"{datetime.now().time()} обработано датасета завершена")
    return df

## Перевод текстов песен

In [7]:
df_lyrics = pd.read_json(PATH_LYRICS, lines=True)
df_meta = pd.read_json(PATH_META, lines=True, convert_dates=['dttm'])[['track_id', 'language']]
df_union = df_meta.merge(df_lyrics, on = 'track_id', how = 'inner')

In [9]:
new_df = translate_df(df_union, 'text', 'en', api_key)

INFO:root:01:29:48.618686 начат перевод датасета
INFO:root:01:29:48.626686 обработано 1 строк
INFO:root:01:36:46.785832 обработано 501 строк
INFO:root:01:46:37.089815 обработано 1001 строк
INFO:root:01:57:17.046477 обработано 1501 строк
INFO:root:02:04:04.300459 обработано 2001 строк
INFO:root:02:11:09.127583 обработано 2501 строк
INFO:root:02:17:42.224014 обработано 3001 строк
INFO:root:02:23:28.278499 обработано 3501 строк
INFO:root:02:30:28.842817 обработано 4001 строк
INFO:root:02:39:36.872441 обработано 4501 строк
INFO:root:02:45:50.468707 обработано 5001 строк
INFO:root:02:47:12.030429 обработано 5501 строк
INFO:root:02:49:22.282235 обработано 6001 строк
INFO:root:02:49:59.038923 обработано 6501 строк
INFO:root:02:51:45.163543 обработано 7001 строк
INFO:root:02:54:26.194776 обработано 7501 строк
INFO:root:02:57:17.139227 обработано 8001 строк
INFO:root:03:01:04.387603 обработано 8501 строк
INFO:root:03:05:34.924808 обработано 9001 строк
INFO:root:03:10:55.048511 обработано 9486 с

In [10]:
new_df.to_csv(PATH_TO_SAVE_LYRICS, index=False)

В этот момент закончились средства на аккаунте. Пополнили их и продолжли

In [41]:
new_df = pd.read_csv(PATH_TO_SAVE_LYRICS)

In [42]:
df_to_translate = new_df[new_df['translate_text'].isna()].reset_index(drop=True)

In [10]:
df_translated = translate_df(df_to_translate, 'text', 'en', api_key)

INFO:root:10:49:19.222063 начат перевод датасета
INFO:root:10:49:20.213480 обработано 1 строк
INFO:root:10:53:34.499393 обработано 501 строк
INFO:root:10:57:58.252885 обработано 1001 строк
INFO:root:11:00:27.850707 обработано 1501 строк
INFO:root:11:03:43.490748 обработано датасета завершена


Переименуем столбцы language и translate_text в новом датасете, чтобы не было пролем при мёрже

In [43]:
df_translated = df_translated.rename(columns={'language': 'new_language',
                                             'translate_text': 'new_translate_text'})

Смержим два датасета и заполним пропуски в строках, где text равен NaN

In [45]:
new_df = new_df.merge(df_translated, on = ['track_id', 'lyricId', 'text'], how='left')

In [46]:
new_df.loc[new_df['translate_text'].isna(), 'language'] = new_df.loc[new_df['translate_text'].isna(), 'new_language']
new_df.loc[new_df['translate_text'].isna(), 'translate_text'] = new_df.loc[new_df['translate_text'].isna(), 'new_translate_text']

Удалим лишние столбцы и выведем последние 5 строк, чтобы убедиться, что всё отработало корректно

In [48]:
new_df = new_df.drop(['new_language', 'new_translate_text'], axis=1)

In [51]:
new_df.tail(5)

,track_id,language,lyricId,text,translate_text
11409,b58ca1bfa08279d4b8bce744262cdf22,RU,3ab73ba22fdf32722fdc80cb3e7d92c5,"Если бы не ты, кто б меня спасал?\nКто бы успо...","If it wasn't for you, who would save me?\nWho ..."
11410,a73b42b709b63cb642422696b38f8f19,EN,871d37f7d03ad7a786d74d09254b1d31,I heard he sang a good song\nI heard he had a ...,I heard he sang a good song\nI heard he had a ...
11411,a95d976c7342f213a2d50a8c9d6c32a8,EN,c89aaa11054fc3edddc6c694410c93e4,Beyond the horizon of the place we lived when ...,Beyond the horizon of the place we lived when ...
11412,495babc700c0c6b5b8549d49f163d3cb,EN,ac58c9835f8bafbd2dd129e6f6eebbb6,Guess it's true I'm not good at a one night st...,Guess it's true I'm not good at a one night st...
11413,5731fc0a34c663a0c0082c712df15266,RU,00cdce64d0559ea65ec245d05735e2b0,Собери по осколкам моё разбитое сердце\nСбита ...,Pick up the pieces of my broken heart\nI'm con...


Выведем информацию по полученному датасету

In [54]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11414 entries, 0 to 11413
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   track_id        11414 non-null  object
 1   language        11359 non-null  object
 2   lyricId         11414 non-null  object
 3   text            11414 non-null  object
 4   translate_text  11414 non-null  object
dtypes: object(5)
memory usage: 446.0+ KB


Есть небольшое количество пропусков в столбце с языком, но их уже осталось незначительное количество

In [53]:
new_df.to_csv(PATH_TO_SAVE_LYRICS, index=False)

## Перевод заголовков

На этапе моделинга оказалось, что заголовки тоже могут оказаться сильным признаком, поэтому переведём дополнительно заголовки песен и определим для них язык (если такое возможно). Откроем датасет с заголовками.

In [20]:
df_meta = pd.read_json(PATH_META, lines=True, convert_dates=['dttm'])[['track_id', 'language', 'title']]

Переведём заголовки

In [8]:
new_df = translate_df(df_meta, 'title', 'en', api_key)

INFO:root:19:15:24.716869 начат перевод датасета
INFO:root:19:15:24.728381 обработано 1 строк
INFO:root:19:18:53.655826 обработано 501 строк
INFO:root:19:21:18.676445 обработано 1001 строк
INFO:root:19:24:37.761239 обработано 1501 строк
INFO:root:19:27:32.059534 обработано 2001 строк
INFO:root:19:31:10.432916 обработано 2501 строк
INFO:root:19:34:46.671787 обработано 3001 строк
INFO:root:19:38:25.867208 обработано 3501 строк
INFO:root:19:41:43.499858 обработано 4001 строк
INFO:root:19:45:27.167053 обработано 4501 строк
INFO:root:19:49:21.019756 обработано 5001 строк
INFO:root:19:52:47.662543 обработано 5501 строк
INFO:root:19:56:15.403141 обработано 6001 строк
INFO:root:20:00:05.803003 обработано 6501 строк
INFO:root:20:03:22.733352 обработано 7001 строк
INFO:root:20:06:29.669834 обработано 7501 строк
INFO:root:20:10:13.749991 обработано 8001 строк
INFO:root:20:13:45.526251 обработано 8501 строк
INFO:root:20:17:55.227205 обработано 9001 строк
INFO:root:20:22:05.117822 обработано 9501 с

Обработка текста прервалась, посмотрим в чём может быть проблема

In [26]:
df_meta[44434:].head()

,track_id,language,title
44434,None,None,None
44435,eeb69a3cb92300456b6a5f4162093851,EN,Proud Mary
44436,fe7ee8fc1959cc7214fa21c4840dff0a,EN,Fortunate Son
44437,cd89fef7ffdd490db800357f47722b20,ES,La Camisa Negra
44438,995665640dc319973d3173a74a03860c,EN,The Way I Are


Проблема заключается в том, что есть строка без данных и это приводит к ошибке, проверим сколько таких строк в базе.

In [24]:
df_meta[44434:]['title'].isna().sum()

1

Такая строка всего одна, тем не менее добавим обработку данного исключения и переведём остатка текста.

In [27]:
new_df2 = translate_df(df_meta[44434:], 'title', 'en', api_key)

INFO:root:03:59:43.819963 начат перевод датасета
INFO:root:03:59:44.940914 обработано 1 строк
INFO:root:04:00:02.687094 обработано 501 строк
INFO:root:04:01:23.815799 обработано 1001 строк
INFO:root:04:03:09.670130 обработано 1501 строк
INFO:root:04:04:05.981773 обработано 2001 строк
INFO:root:04:05:54.353915 обработано 2501 строк
INFO:root:04:07:53.570744 обработано 3001 строк
INFO:root:04:09:53.456206 обработано 3501 строк
INFO:root:04:11:45.534363 обработано 4001 строк
INFO:root:04:13:15.411467 обработано 4501 строк
INFO:root:04:15:53.353064 обработано 5001 строк
INFO:root:04:17:25.700579 обработано 5501 строк
INFO:root:04:18:50.416794 обработано 6001 строк
INFO:root:04:20:03.863742 обработано 6501 строк
INFO:root:04:21:17.935778 обработано 7001 строк
INFO:root:04:21:57.403352 обработано 7501 строк
INFO:root:04:22:59.349806 обработано 8001 строк
INFO:root:04:23:59.122433 обработано 8501 строк
INFO:root:04:25:02.162762 обработано 9001 строк
INFO:root:04:27:10.075380 обработано 9501 с

Объединим две части датасета

In [46]:
new_df = pd.read_csv(PATH_TO_SAVE_TITLE)

In [47]:
new_df = pd.concat([new_df[:44434], new_df2], axis = 0).reset_index(drop=True)

Переименуем столбцы

In [49]:
new_df = new_df.rename(columns={'language': 'title_language',
                                'translate_text': 'translate_title'})

Посмотрим на полученный результат

In [50]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71769 entries, 0 to 71768
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   track_id         71768 non-null  object
 1   title_language   71689 non-null  object
 2   title            71768 non-null  object
 3   translate_title  71768 non-null  object
dtypes: object(4)
memory usage: 2.2+ MB


Сохраним полученный датасет

In [51]:
new_df.to_csv(PATH_TO_SAVE_TITLE, index=False)

## Выводы 

В данном разделе были решены 2 задачи:
1. Все тексты были переведены на английский язык, что значительно упростит задачу очистки текстов и их векторизацию
2. Почти для всех тестов был определён язык на котором они написаны, а это может стать сильным признаком для модели машинного обучения.

Для решения данных задач использовался Yandex Translator, с которым мы работали через API запросы ри помощи модуля request.